In [17]:
import pandas as pd
import openpyxl
from pandas import ExcelWriter

In [18]:
%store -r tarjetas
%store -r bancos

In [19]:
tarjeta_nativa = tarjetas.loc [tarjetas['Cod Marca Producto']== 'Mastercard'] #Filtro por la Tarjeta Mastercard
tarjeta_nativa = tarjeta_nativa.drop(tarjeta_nativa.index[(tarjeta_nativa["Cod Comercio"] == 4547438)],axis=0) # Sacar ese Cod de Comercio
tarjeta_nativa = tarjeta_nativa.loc[:, (tarjeta_nativa!= 0).any(axis=0)]

In [16]:
a = tarjeta_nativa.loc[tarjeta_nativa['PERCEPCION IVA R.G. 2408  1,50/3,00 %'] > 0]
a.loc[:, (a != 0).any(axis=0)]

Desc Concepto Clearing,Fecha de Pago,Cod Marca Producto,Cod Comercio,Nro Liquidacion,"PERCEPCION IVA R.G. 2408 3,00 %",ARANCEL,CARGO SISTEMA CUOTAS MENS,DESCUENTO FINANC.OTORG. CUOTAS,IMPORTE NETO DE PAGOS,"IVA CRED.FISC.COM.L.25063 S/DTO F.OTOR 10,50",...,PROMO CUOTAS AHORA 12/18,RETENCION IMP.GANANCIAS,RETENCION ING.BRUTOS,RETENCION ING.BRUTOS SIRTAC,RETENCION IVA,TOTAL DEDUCCIONES,TOTAL PAGOS DE COMERCIOS,VENTAS C/DESCUENTO CONTADO,VENTAS C/DTO CUOTAS FINANC. OTORG.,"PERCEPCION IVA R.G. 2408 1,50/3,00 %"
8,2022-11-01,Mastercard,15537297,89890,111.67,3722.41,0.0,0.00,175821.18,0.00,...,15379.97,1876.99,0.00,1861.21,5630.96,30979.82,0.00,3899.0,202902.00,111.67
184,2022-11-02,Mastercard,15537297,89920,91.33,3043.93,0.0,0.00,144647.97,0.00,...,11757.71,1543.07,0.00,1521.98,4629.22,24461.03,0.00,13994.0,155115.00,91.33
365,2022-11-03,Mastercard,15537297,89944,155.32,5177.12,0.0,0.00,246819.80,0.00,...,19244.33,2632.00,0.00,2588.59,7895.99,40801.20,0.00,33738.0,253883.00,155.32
529,2022-11-04,Mastercard,15537297,89969,219.89,7329.68,0.0,0.00,346351.44,0.00,...,30146.34,3697.30,0.00,3664.85,11091.90,60854.56,0.00,9497.0,397709.00,219.89
714,2022-11-07,Mastercard,15537297,90000,427.94,14265.39,0.0,0.00,673867.14,0.00,...,58878.28,7193.81,0.00,7132.72,21581.43,118657.52,0.00,36392.0,756132.66,427.94
719,2022-11-07,Mastercard,15550922,90005,70.08,2335.84,0.0,0.00,107788.37,0.00,...,9836.49,1175.97,2257.86,1167.92,3527.90,21980.63,0.00,0.0,129769.00,70.08
720,2022-11-07,Mastercard,15550935,90006,72.11,2403.77,0.0,0.00,110925.54,0.00,...,10119.83,1210.19,2323.56,1201.88,3630.57,22616.95,0.00,7898.0,125644.49,72.11
725,2022-11-07,Mastercard,15554071,90011,76.06,2534.73,0.0,0.00,119508.85,0.00,...,10674.04,1276.10,0.00,1267.37,3828.29,21309.65,0.00,0.0,140818.50,76.06
726,2022-11-07,Mastercard,15554084,90012,82.27,2742.39,0.0,0.00,127920.36,0.00,...,12844.12,1367.69,0.00,1371.20,4103.07,24435.27,0.00,2000.0,150355.63,82.27
728,2022-11-07,Mastercard,15554106,90014,125.06,4168.30,0.0,0.00,192109.51,0.00,...,16962.22,2104.43,5050.64,2084.17,6313.30,39464.49,0.00,7798.0,223776.00,125.06


In [20]:
tarjeta_nativa["PERCEPCION IVA R.G. 2408  1,50/3,00 %"] = (tarjeta_nativa['PERCEPCION IVA R.G. 2408   1,50 %'] 
                                                          + tarjeta_nativa['PERCEPCION IVA R.G. 2408   3,00 %'])
tarjeta_nativa['VENTAS'] = tarjeta_nativa['VENTAS C/DESCUENTO CONTADO'] + tarjeta_nativa['VENTAS C/DTO CUOTAS FINANC. OTORG.']
tarjeta_nativa['RETENCION ING.BRUTOS SIRTAC'] = (tarjeta_nativa['RETENCION ING.BRUTOS SIRTAC']
                                              - tarjeta_nativa['AJUSTE SIRTAC NOV. 1RA QU'])
                                                                     
tarjeta_nativa["TOTAL DEDUCCIONES"] = tarjeta_nativa[["TOTAL DEDUCCIONES", "TOTAL PAGOS DE COMERCIOS"]
                                                             ].sum(axis=1, min_count=1)
tarjeta_nativa["TOTAL DEDUCCIONES"]= tarjeta_nativa["TOTAL DEDUCCIONES"] - tarjeta_nativa['REEMBOLSOS DE ENTIDAD']
tarjeta_nativa['IVA 21%'] = tarjeta_nativa[['IVA CRED.FISC.COMERCIO S/ARANC 21,00%','IVA PROMO CUOTAS AHORA 12/18 - 21,00%'
                              , 'IVA RI SIST CUOTAS']].sum(axis=1, min_count=1)
tarjeta_nativa['ARANCEL'] = tarjeta_nativa['ARANCEL'] + tarjeta_nativa['CARGO  SISTEMA CUOTAS MENS']
tarjeta_nativa['DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18'] = (tarjeta_nativa['DESCUENTO FINANC.OTORG. CUOTAS']
                                                                              + tarjeta_nativa['PROMO CUOTAS AHORA 12/18'])
tarjeta_nativa['Iva 10,5%'] = (tarjeta_nativa['IVA PROMO CUOTAS AHORA 12/18 - 10,50%'] 
                               + tarjeta_nativa['IVA CRED.FISC.COM.L.25063 S/DTO F.OTOR 10,50'])
tarjeta_nativa = tarjeta_nativa.reindex(columns=['Fecha de Pago', 'Cod Marca Producto', 'Cod Comercio',
       'Nro Liquidacion','VENTAS', 'ARANCEL', 'IVA 21%', 'DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18',
       'Iva 10,5%', 'RETENCION IVA','PER B.A.I.BR.DN.01/04', 'RETENCION IMP.GANANCIAS','RETENCION ING.BRUTOS',
       'RETENCION ING.BRUTOS SIRTAC','PERCEPCION IVA R.G. 2408  1,50/3,00 %',
       'TOTAL DEDUCCIONES', 'IMPORTE NETO DE PAGOS']) #reordeno columnas, aca ya quedan en el orden que es necesario
tarjeta_nativa['DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18'] = tarjeta_nativa['Iva 10,5%']*100/10.5
tarjeta_nativa['ARANCEL'] = tarjeta_nativa['IVA 21%']*100/21            

tarjeta_nativa.insert(7, 'Iva 21 control', tarjeta_nativa['IVA 21%']/tarjeta_nativa['ARANCEL'])
                                #agrego columna de control para IVA 21%
tarjeta_nativa.insert(10, 'Iva 10,5 control', 
                                tarjeta_nativa['Iva 10,5%']/tarjeta_nativa['DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18'])
                                #agrego columna de control para IVA 10,5%                                
tarjeta_nativa = tarjeta_nativa.assign(Diferencia = tarjeta_nativa.iloc[:, 5:7].sum(axis=True) 
                         + tarjeta_nativa.iloc[:, 8:10].sum(axis=True)
                         + tarjeta_nativa.iloc[:, 11:17].sum(axis=True)
                         - tarjeta_nativa['TOTAL DEDUCCIONES']) #agrego columna de control final
tarjeta_nativa = tarjeta_nativa.round(2)

In [24]:
sum_columns = [
       'Nro Liquidacion', 'VENTAS', 'ARANCEL', 'IVA 21%', 'Iva 21 control',
       'DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18', 'Iva 10,5%',
       'Iva 10,5 control', 'RETENCION IVA', 'PER B.A.I.BR.DN.01/04',
       'RETENCION IMP.GANANCIAS', 'RETENCION ING.BRUTOS',
       'RETENCION ING.BRUTOS SIRTAC', 'PERCEPCION IVA R.G. 2408  1,50/3,00 %',
       'TOTAL DEDUCCIONES',
       'IMPORTE NETO DE PAGOS', 'Diferencia']
max_columns = ['Nro Liquidacion']

group = {col: ('max' if col in max_columns else 'sum') for col in sum_columns}



In [25]:
tarjeta_nativa=tarjeta_nativa.groupby(by=['Cod Marca Producto', 'Cod Comercio']).agg(group)

In [26]:
tarjeta_nativa = tarjeta_nativa.reset_index()

In [27]:
tarjeta_nativa = tarjeta_nativa.drop(['Diferencia', 'Iva 21 control', 'Iva 10,5 control'], axis=1)

In [28]:
writer = ExcelWriter(r'C:\Users\Juan M Cabral\Desktop\Liquidaciones tarjetas\11-22\Nativa 11-22.xlsx')
tarjeta_nativa.to_excel(writer, 'Hoja de datos', index=False)
writer.save()

C:\Users\Juan M Cabral\AppData\Local\Temp\ipykernel_11812\21504442.py:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [22]:
tarjeta_nativa['Diferencia'].value_counts()

 0.00    85
 0.02    67
-0.02    65
-0.01    54
 0.03    53
 0.01    52
-0.03    52
-0.04    33
 0.04    24
 0.05    23
-0.05    21
-0.06    19
 0.06    15
 0.07     3
 0.08     2
Name: Diferencia, dtype: int64